# PSO Examples

This script is mainly to demostrate the pso (particle swarm optimization) function.

For the steps and theories of the pso function building, please refer the documentation at **README.md** and the codes in **pso_fun.py**

# Part I. Basics

In [2]:
import numpy as np
from pso_fun import pso_min

Assume the function to optimize (find minimum value in domain) is
$$ y = -(x_1^2 + x_2^2+ x_3^2+ x_4^2) $$ where all x $\in$ [0, 30]

It is easy to know the optimum value is **-3600** when **x = [30,30,30,30]**

In [3]:
def test_fun_1(x):
    x1, x2, x3, x4 = x
    y = -(x1 ** 2 + x2 ** 2 + x3 ** 2 + x4 ** 2)
    return y

lower_bound = [0, 0, 0, 0]
upper_bound = [30, 30, 30, 30]

In [4]:
pso_min(test_fun_1, lower_bound, upper_bound, random_seed=2021, show_result=True)
# here we set the random seed to 233 for reproducible test

Achieve the minimum difference of 1e-08 at 33 iterations.
The achieved optimum (minimum) value is -3599.99
The corresponding solution is [30. 30. 30. 30.]


(-3599.993711037324,
 array([29.99999413, 29.99999738, 29.99997519, 29.99992848]))

Now let's change the domain:

In [5]:
new_lower_bound = [-25.5, 0, np.pi, 0]
new_upper_bound = [35.5, 60, 35.5, 30]

In [6]:
pso_min(test_fun_1, new_lower_bound, new_upper_bound, random_seed=2021, show_result=True)

Achieve the minimum difference of 1e-08 at 31 iterations.
The achieved optimum (minimum) value is -7020.46
The corresponding solution is [35.5 60.  35.5 30. ]


(-7020.459729352235,
 array([35.49976432, 59.99999537, 35.49968024, 29.99999536]))

The function also provided the resonalbe solution and value.

Now let's add some **constraints** on this problem:
$$ x_1 > 2x_2 $$
$$ x_3^2 > x_4^3 + 5 $$

Transfer the constraints into coding:

In [7]:
def constraints(x):
    x1, x2, x3, x4 = x
    return [x1 - 2*x2, x3**2 - x4**3 - 5]

In [8]:
pso_min(test_fun_1, lower_bound, upper_bound, cons=constraints, 
        random_seed=2021, show_result=True)

Achieve the minimum difference of 1e-08 at 21 iterations.
The achieved optimum (minimum) value is -2015.11
The corresponding solution is [29.98 14.81 29.99  0.  ]


(-2015.1086357892452,
 array([2.99844813e+01, 1.48117257e+01, 2.99903859e+01, 4.69984190e-03]))

The function also performed well in this situation.

# Part II. Benchmark Testing

The Griewank funciton has been choosed for algorithm testing, which is:
$$ f(x) = \sum_{i+1}^d(\frac{x_i^2}{4000}) - \prod_{i=1}^d \cos(\frac{x_i}{\sqrt{i}}) + 1$$

In this testing, $ -500<x_i<500 $ where $d=10$ and $x=(x_1, x_2, ..., x_{10})$.

The function is a highly multimodel function, and its global optimum value is $f(x)=0$ when $x_1=x_2=x_3=...=x_{10}=0$.

In [11]:
def test_sum_term(x):
    x1,x2,x3,x4,x5,x6,x7,x8,x9,x10 = x
    y = (x1**2 + x2**2 + x3**2 + x4**2 + x5**2 + x6**2 + x7**2 + x8**2 + x9**2 + x10**2)/4000
    return y

def test_product_term(x):
    x1,x2,x3,x4,x5,x6,x7,x8,x9,x10 = x
    y = np.cos(x1/np.sqrt(1)) * np.cos(x2/np.sqrt(2)) * np.cos(x3/np.sqrt(3)) * np.cos(x4/np.sqrt(4)) * \
    np.cos(x5/np.sqrt(5)) * np.cos(x6/np.sqrt(6)) * np.cos(x7/np.sqrt(7)) * np.cos(x8/np.sqrt(8)) * \
    np.cos(x9/np.sqrt(9)) * np.cos(x10/np.sqrt(10))
    return y

def benchmark_fun(x):
    y = test_sum_term(x) - test_product_term(x) + 1
    return y

In [12]:
bf_lower_bound = [-500] * 10
bf_upper_bound = [500] * 10

In [17]:
pso_min(benchmark_fun, bf_lower_bound, bf_upper_bound, 
        swarm_size=500, gen_max=500,
        random_seed=2021, show_result=True)

Achieve the minimum difference of 1e-08 at 101 iterations.
The achieved optimum (minimum) value is 2.05
The corresponding solution is [ -8.85   8.26  -0.75  36.3   -6.27 -19.68 -11.84 -28.77  29.21 -22.72]


(2.0482106933292545,
 array([ -8.85101945,   8.26159229,  -0.75140436,  36.3002936 ,
         -6.26837869, -19.67657442, -11.8446883 , -28.7676104 ,
         29.21336343, -22.71732005]))

The pso get an acceptable value (~2.05) at the 101 iteration.

# Part III. Model Introduction

This function is mainly created based on two PSO models from:

1. **An improved Particle Swarm Optimization with Revivable Leaders and Its Application in Constrained NP Optimization** (*Harry Liang*)
2. **A Chaos-Enhanced Particle Swarm Optimization with Adaptive Parameters and Its Application in Maximum Power Point Tracking** (*Ying-Yi Hong*)

Please refer the source for reference at this link: https://www.hindawi.com/journals/mpe/2016/6519678/#introduction

The mathematical equation is represented as follows:
$$
v_p^{t+1} = \chi \times \omega_{chaos}^t \times v_p^t + c_1^t \times(p_{best}^t-x_p^t) + c_2^t \times(g_{best}^t-x_p^t)
$$

$$
x_p^{t+1} = x_p^t+v_p^{t+1}
$$

Where $ \chi $ is Type 1'' constriction coefficient which is integrated to the proposed variant of PSO to prevent the divergence of the particles:
$$
\chi=\frac{2}{(\phi-2+\sqrt{\phi^2-4\phi})}
$$

$$
\phi=c_1^t+c_2^t
$$

and $ \omega_{chaos}^t $ is the chaotic random inertia weight, which is given by
$$
\omega_{chaos}^t = 0.5 \times rand(.) + 0.5 \times z_{t+1}
$$

$$
z_t+1=\left|\sin\left(\frac{\pi z_t}{rand(.)}\right) \right|
$$

$ c_1^t, c_2^t $ are time-varying cognitive and social parameters which are incorporated into PSO to improve its local and the global search by making the cognitive component large and the social component small at the initialization or in the early part of the evolutionary process.

$$
c_1^t = c_{1,f}-\frac{t}{MAXITR}(c_{1,f}-c_{1,i}) 
$$

$$
c_2^t = c_{2,i}+\frac{t}{MAXITR}(c_{2,f}-c_{2,i}) 
$$


where $c_{1,i}, c_{2,i}, c_{1,f}$, and $c_{2,f} $ are the initial and final values of the cognitive parameters and the social parameters, respectively; $MAXITR$ is the current generation, and the  is the value in the final generation.